In [1]:

# %reload_ext autoreload
# %autoreload 2

# import os
# import sys

# # Add the root folder to the module search path
# # Get the current directory
# current_directory = os.getcwd()

# # Move two levels up (go to the parent directory of the parent directory)
# # two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))
# one_levels_up_directory = os.path.dirname(current_directory)

# print(one_levels_up_directory)

# # sys.path.append(two_levels_up_directory)
# sys.path.append("/home/ubuntu/git/pykoi/pykoi")


In [2]:
import pykoi

/opt/conda/envs/pykoi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define my DB

In [3]:
qa_database = pykoi.QuestionAnswerDatabase()

### Insert my data to the DB

In [4]:
import pandas as pd
my_stackoverflow_dataset = pd.read_json("rlhf_qa_dataset.json", orient="records")
my_stackoverflow_dataset

,Question,Answer
0,What is InstructGPT?,InstructGPT is a language model developed by O...
1,Why does InstructGPT work?,InstructGPT works due to a two-step training p...
2,What are some commonly used evaluation metrics...,One main evaluation metric for InstructGPT is ...
3,How is InstructGPT used?,InstructGPT can be used in any application tha...
4,What are some common applications of InstructGPT?,Common applications of InstructGPT can be in e...
5,How does InstructGPT handle ambiguous prompts?,"For ambiguous prompts, InstructGPT aims to ask..."
6,Can InstructGPT generate incorrect or nonsensi...,"Yes, InstructGPT can sometimes produce plausib..."
7,How does InstructGPT manage harmful and biased...,InstructGPT has a moderation system in place t...
8,What is the role of human evaluators in the tr...,Human evaluators play a crucial role in the tr...
9,What are the limitations of InstructGPT?,There are several limitations to InstructGPT. ...


In [4]:
QA_CSV_HEADER_ID = 'ID'
QA_CSV_HEADER_QUESTION = 'Question'
QA_CSV_HEADER_ANSWER = 'Answer'
QA_CSV_HEADER_VOTE_STATUS = 'Vote Status'
QA_CSV_HEADER_TIMESTAMPS = 'Timestamp'
QA_CSV_HEADER = (
    QA_CSV_HEADER_ID,
    QA_CSV_HEADER_QUESTION,
    QA_CSV_HEADER_ANSWER,
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

In [ ]:
for row in my_stackoverflow_dataset.iloc[3:100].to_dict('records'):
    qa_id = qa_database.insert_question_answer(question=row[QA_CSV_HEADER_QUESTION],
                                       answer=row[QA_CSV_HEADER_ANSWER])
    qa_database.update_vote_status(id=qa_id, vote_status="up") #row[QA_CSV_HEADER_VOTE_STATUS])

In [5]:
qa_database

### Train RLHF using the data from database

Let's take a look of the QA data and process it for training.

In [6]:
my_data_pd = qa_database.retrieve_all_question_answers_as_pandas()
my_data_pd.head()

,ID,Question,Answer,Vote Status,Timestamp
0,1,who founded YC and what does YC do,Y Combinator is a startup accelerator that pro...,down,2023-07-24 08:28:29.461045
1,2,what are the top YC companies,I am looking for a list of the top YC companie...,down,2023-07-24 08:29:17.521467
2,3,How much does top 10 YC companies worth,"According to the latest Crunchbase data, the 1...",down,2023-07-24 08:29:58.248998
3,4,What's the meaning of life,"The meaning of life is to be happy, to be usef...",down,2023-07-24 19:26:04.713297
4,5,What is the meaning of life,The meaning of life is to find your gift. The ...,up,2023-07-24 19:26:28.676427


In [7]:
my_data_pd = my_data_pd[my_data_pd[QA_CSV_HEADER_VOTE_STATUS]=="up"]
my_data_pd.shape

(118, 5)

In [8]:
from datasets import Dataset

my_data_pd = my_data_pd[[QA_CSV_HEADER_ID,
                        QA_CSV_HEADER_QUESTION,
                        QA_CSV_HEADER_ANSWER]]
print("My local database has {} samples".format(my_data_pd.shape[0]))
dataset = Dataset.from_dict(my_data_pd)
dataset

My local database has 118 samples


Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 118
})

### Train with RLHF

In [10]:
# run supervised finetuning
config = pykoi.RLHFConfig(base_model_path="meta-llama/Llama-2-7b-hf", dataset_type="local_db")
rlhf_step1_sft = pykoi.SFT(config)
rlhf_step1_sft.train_and_save("./models/rlhf_step1_sft")


/opt/conda/envs/pykoi/lib/python3.10/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


My local database has 118 samples
Size of the train set: 106.               Size of the validation set: 12


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.71s/it]
/opt/conda/envs/pykoi/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/opt/conda/envs/pykoi/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/envs/pykoi/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(
/opt/conda/envs/pykoi/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed i

Step,Training Loss,Validation Loss


: 